# NNTI Assignment

@authors:  Mhd Jawad Al Rahwanji , Christian Singer
@ID:  7038980, 7039059
@email: mhal00002@stud.uni-saarland.de, chsi00002@stud.uni-saarland.de

## Exercise 1

### Let $A\in\mathbb{R}^{m\times n}, B\in\mathbb{R}^{n\times p}$ and $B\in\mathbb{R}^{p\times q}$ then $L\equiv AB\in\mathbb{R}^{m\times p}\text{ and }S\equiv (AB)C = RC\in\mathbb{R}^{m\times q}$
### with $r_{i k}=\sum_{l=1}^n a_{i l} \cdot b_{l k}$ and $s_{i j}=\sum_{k=1}^p r_{i k} \cdot c_{k j}$, using the distributive law for addition and multiplication this yields
### $s_{i j}=\sum_{k=1}^p (\sum_{l=1}^n a_{i l} \cdot b_{l k}) \cdot c_{k j} = \sum_{k=1}^p \sum_{l=1}^n (a_{i l} \cdot b_{l k}) \cdot c_{k j} = \sum_{k=1}^p \sum_{l=1}^n a_{i l} (\cdot b_{l k} \cdot c_{k j}) = \sum_{l=1}^n \sum_{k=1}^p a_{i l} (\cdot b_{l k} \cdot c_{k j})$
### which shows the associativity of matrix multiplication.


## Exercise 3

In [1]:
# Import necessary libraries
import os
from typing import List, Tuple

import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image

In [2]:
# Custom dataset class for loading image datasets.
class CustomImageDataset(Dataset):
    """
    Create a pytorch dataset containing images with corresponding labels.
    """
    def __init__(self, labels_dir: str, img_dir: str):
        super().__init__()
        self.img_dir = img_dir
        self.img_labels = pd.read_csv(labels_dir, header=None)

    def __len__(self) -> int:
        return len(self.img_labels)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        return image, label

# Custom DataLoader to load our dataset.
class CustomImageDataLoader(DataLoader):
    """
    Create a pytorch dataloader for the custom image dataset.
    """

    def __init__(self, labels_dir: str, img_dir: str, batch_size: int):
        self.dataset = CustomImageDataset(labels_dir, img_dir)
        super().__init__(
            self.dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn
        )

In [3]:
# Define collate function
def collate_fn(batch: List[torch.Tensor]) -> Tuple[torch.Tensor, List[int]]:
    """
    Resize each axis of the image, such that the total image is 20% smaller.
    """
    # https://stackoverflow.com/questions/29139350/difference-between-ziplist-and-ziplist
    images, labels = zip(*batch)
    # Unsqueezing the images simulates a minibatch of size 1.
    images = [
        F.interpolate(image.unsqueeze(0), scale_factor=pow(0.8, 0.5)).squeeze(0)
        for image in images
    ]
    return torch.stack(images), labels

In [4]:
# The code in the cells below lets you verify that our implementations fulfill the criteria specified on the exercise sheet.
dataset = CustomImageDataset(labels_dir="data\\labels.csv", img_dir="data\\images")

# Proof that dataset has a length.
print(f"Length of dataset: {len(dataset)}")

# Proof that dataset is iterable.
for i, x in enumerate(dataset):
    print(f"Size of image {i}: {x[0].shape}, label: {x[1]}")

# Proof that dataset is a valid input to DataLoader.
dataloader = CustomImageDataLoader(labels_dir="data\\labels.csv", img_dir="data\\images", batch_size=4)
image, label = next(iter(dataloader))
print(f"Labels: {label}")
# Proof that images have been resized to 80% of their original size.
print(f"Size of image batch: {image.shape}")

Length of dataset: 10
Size of image 0: torch.Size([3, 512, 512]), label: 0
Size of image 1: torch.Size([3, 512, 512]), label: 0
Size of image 2: torch.Size([3, 512, 512]), label: 0
Size of image 3: torch.Size([3, 512, 512]), label: 0
Size of image 4: torch.Size([3, 512, 512]), label: 0
Size of image 5: torch.Size([3, 512, 512]), label: 1
Size of image 6: torch.Size([3, 512, 512]), label: 1
Size of image 7: torch.Size([3, 512, 512]), label: 1
Size of image 8: torch.Size([3, 512, 512]), label: 1
Size of image 9: torch.Size([3, 512, 512]), label: 1
Labels: (1, 0, 0, 1)
Size of image batch: torch.Size([4, 3, 457, 457])
